<a href="https://colab.research.google.com/github/Droslj/scATAC-seq-complete-/blob/Google-colab/scATAC_seq_(2)_DA_diffxpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scATAC seq, based on scATAC seq processing Galaxy tutorials (scATAC preprocessing (2), Standard scATAC seq processing pipeline (1) )
AD Objects created in Galaxy using customized Galaxy WF with Snapatac2 and imported
(1) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-preprocessing-tenx/tutorial.html#mapping-reads-to-a-reference-genome, (2) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-standard-processing-snapatac2/tutorial.html
Data taken from the following NCBI study:
Metabolic adaptation pilots the differentiation of human hematopoietic cells (https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1015713)
Import Anndata objects for two biological replicates, SRR26046013 (cells treated with AOA inhibitor) and SRR26046019 (untreated cells)
Perform following steps:
(1) Import matrices
(2) Compute fragment size distribution
(3) Compute TSS enrichment
(4) Filter cell counts based on TSSe
(5) Create cell by bin matrix based on 500 bp wide bins accross the whole genome
(6) Perform feature selection
(7) Perform Doublet removal
(8) Perform Dim reduction (spectral)
(9) Perform Clustering (neighborhood, UMAP, leiden)
(10) Create a cell by gene matrix
(11) Concatenate matrices using Inner join
(12) Remove batch effects

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab

In [ ]:
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda --version

conda 23.11.0


In [ ]:
!which conda

/usr/local/bin/conda


In [ ]:
!conda config --add channels conda-forge

In [ ]:
!conda config --add channels bioconda

In [ ]:
!pip install snapatac2 -q

In [ ]:
!pip show snapatac2

Name: snapatac2
Version: 2.8.0
Summary: SnapATAC2: Single-cell epigenomics analysis pipeline
Home-page: https://github.com/
Author: Kai Zhang <kai@kzhang.org>
Author-email: Kai Zhang <zhangkai33@westlake.edu.cn>
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: anndata, igraph, kaleido, macs3, multiprocess, natsort, numpy, pandas, plotly, polars, pooch, pyarrow, pyfaidx, rustworkx, scikit-learn, scipy, tqdm, typeguard
Required-by: 


In [ ]:
import snapatac2 as snap

In [ ]:
!pip install umap-learn



In [ ]:
import umap.umap_ as umap


In [ ]:
from umap import UMAP

In [ ]:
!pip install scanpy -q

In [ ]:
import scanpy as sc

In [ ]:
pip show scanpy

Name: scanpy
Version: 1.10.4
Summary: Single-Cell Analysis in Python.
Home-page: 
Author: Alex Wolf, Philipp Angerer, Fidel Ramirez, Isaac Virshup, Sergei Rybakov, Gokcen Eraslan, Tom White, Malte Luecken, Davide Cittaro, Tobias Callies, Marius Lange, Andrés R. Muñoz-Rojas
Author-email: 
License: 
Location: /usr/local/lib/python3.10/site-packages
Requires: anndata, h5py, joblib, legacy-api-wrap, matplotlib, natsort, networkx, numba, numpy, packaging, pandas, patsy, pynndescent, scikit-learn, scipy, seaborn, session-info, statsmodels, tqdm, umap-learn
Required-by: 


In [ ]:
import numpy as np

In [ ]:
import anndata as ad

In [ ]:
!pip install diffxpy -q

In [ ]:
import diffxpy.api as de

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go

In [ ]:
from scipy import stats

In [ ]:
import pandas as pd

# Import reads from google drive, three samples treated with energy metabolism inhibitors and one untreated

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Load AD matrix from google drive, PCA and Batch corrected
adata_concat = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/MTXmerged_PCA_BC.h5ad')

/usr/local/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


# Make observation names unique



In [ ]:
adata_concat

AnnData object with n_obs × n_vars = 21361 × 53017
    obs: 'n_fragment', 'frac_dup', 'frac_mito', 'tsse', 'doublet_probability', 'doublet_score', 'leiden', 'Treatment'
    uns: 'pca'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'

In [ ]:
# Check if the number of unique gene names is equal to the total number of gene names
is_unique = len(adata_concat.var_names) == pd.Series(adata_concat.var_names).nunique()

if is_unique:
    print("var_names are unique.")
else:
    print("var_names are not unique.")

var_names are unique.


In [ ]:
adata_concat.var_names

Index(['DDX11L1', 'WASH7P', 'ENSG00000238009', 'ENSG00000239945', 'DDX11L17',
       'WASH9P', 'ENSG00000228463', 'ENSG00000286448', 'U6', 'ENSG00000230021',
       ...
       'MT-ND4', 'MT-TH', 'MT-TS2', 'MT-TL2', 'MT-ND5', 'MT-ND6', 'MT-TE',
       'MT-CYB', 'MT-TT', 'MT-TP'],
      dtype='object', length=53017)

In [ ]:
experiment_names = ["Treated w/AOA", "Treated w/DON", "Treated w/DG", "Untreated"]

# Create a new column to store combined barcode and treatment information
adata_concat.obs['barcode_treatment'] = adata_concat.obs_names.astype(str)

# Modify the 'barcode_treatment' column
for i, treatment in enumerate(experiment_names):
    cell_indices = adata_concat.obs.index[adata_concat.obs["Treatment"] == treatment]
    adata_concat.obs.loc[cell_indices, 'barcode_treatment'] = adata_concat.obs.loc[cell_indices, 'barcode_treatment'] + "_" + str(i + 1)

/usr/local/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
adata_concat.obs

,n_fragment,frac_dup,frac_mito,tsse,doublet_probability,doublet_score,leiden,Treatment
AAACAACGAACGAGCA,20919,0.502426,0.000096,25.061425,0.052336,0.047059,7,Treated w/AOA
AAACAACGAAGAGGCT,13308,0.488184,0.000826,23.198793,0.066163,0.009174,4,Treated w/AOA
AAACAACGAAGTCGGA,17302,0.502830,0.000058,18.234291,0.061446,0.015294,5,Treated w/AOA
AAACAACGAGAAGGCT,14109,0.530246,0.000071,29.930591,0.070621,0.004525,0,Treated w/AOA
AAACAACGAGTACGTT,12165,0.588867,0.000000,21.409091,0.062457,0.013841,7,Treated w/AOA
...,...,...,...,...,...,...,...,...
TTTGGGATGAACGTAA,33918,0.336626,0.001766,18.594706,0.185808,0.043189,1,Untreated
TTTGGGATGAAGTCTC,25630,0.307834,0.000117,15.581623,0.221046,0.025360,7,Untreated
TTTGGGATGATGTGGG,28438,0.336774,0.001019,19.425019,0.189493,0.040388,0,Untreated
TTTGGGATGCTCACGG,20103,0.336676,0.000149,26.203209,0.265871,0.012835,4,Untreated


# Differential accessibility analysis using diffxpy

In [ ]:
adata_concat.obs_names

Index(['AAACAACGAACGAGCA_1', 'AAACAACGAAGAGGCT_1', 'AAACAACGAAGTCGGA_1',
       'AAACAACGAGAAGGCT_1', 'AAACAACGAGTACGTT_1', 'AAACACGCTATCCTTT_1',
       'AAACACGCTCAGTCTA_1', 'AAACACGCTCATACTG_1', 'AAACACGCTCCAAACT_1',
       'AAACACGCTCTACTCC_1',
       ...
       'TTTGGGAACCGACATT_4', 'TTTGGGAACCGAGTGA_4', 'TTTGGGAACGAAGTTA_4',
       'TTTGGGAACTGACCTA_4', 'TTTGGGATGAACGCAG_4', 'TTTGGGATGAACGTAA_4',
       'TTTGGGATGAAGTCTC_4', 'TTTGGGATGATGTGGG_4', 'TTTGGGATGCTCACGG_4',
       'TTTGGGATGCTCTGGG_4'],
      dtype='object', name='barcode_treatment', length=21361)

In [ ]:
# Check if any gene names are duplicated
has_duplicates = adata_concat.var_names.duplicated().any()

if has_duplicates:
    print("var_names are not unique.")
else:
    print("var_names are unique.")

var_names are unique.


In [ ]:
# Check if the number of unique observations is equal to the total number of observations
is_unique = len(adata_concat.obs_names) == pd.Series(adata_concat.obs_names).nunique()

if is_unique:
    print("Observations are unique.")
else:
    print("Observations are not unique.")

Observations are not unique.


In [ ]:
# Check if any observations are duplicated
has_duplicates = adata_concat.obs_names.duplicated().any()

if has_duplicates:
    print("Observations are not unique.")
else:
    print("Observations are unique.")

Observations are not unique.


In [ ]:
#numpy type aliases
np.float = float
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool    #module 'numpy' has no attribute 'bool'

In [ ]:
import dask.array as da

In [ ]:
import scipy.sparse as sparse

In [ ]:
#Optionally set 'barcode_treatment' as the new index:
adata_concat.obs_names = pd.Index(adata_concat.obs['barcode_treatment'])

In [ ]:
adata_concat

AnnData object with n_obs × n_vars = 21361 × 53017
    obs: 'n_fragment', 'frac_dup', 'frac_mito', 'tsse', 'doublet_probability', 'doublet_score', 'leiden', 'Treatment', 'barcode_treatment'
    uns: 'pca'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'

# Run diffxpy wald test

In [ ]:
# Subsample features (e.g., top 1000 highly variable genes)
adata_subset = adata_concat[:, adata_concat.var['highly_variable'][:1000].index]

# Then, perform the Wald test on the subset:
test = de.test.wald(
    data=adata_subset,
    formula_loc="~ 1 + Treatment",
    factor_loc_totest="Treatment"
)